In [ ]:
#| default_exp src.analysis

# Analysis
> Group of tools for analyzing the data

In [ ]:
#| export
from os import getcwd
from os.path import isfile
from shutil import copyfile
from pathlib import Path
from datetime import datetime
from fastcore.script import *
from fastcore.basics import patch
from nbdev import *

from HighResAnalysis.plotting.save import *
from HighResAnalysis.plotting.utils import warning, Config, choose, info, add_to_info, GREEN, RED
from HighResAnalysis.utility.utils import Dir, print_banner, byte2str, ensure_dir

Welcome to JupyROOT 6.28/00


In [ ]:
#| export
class BeamTest:
    """ structure containing information about a beam test:
        - Path to the data Dir
        - Location
        - T - date as a datetime
        - Year
        - Tag and Name - same as T but in string formats"""
    def __init__(self, 
                 p: Path # path to the data dir
                ):

        self.Path = p
        self.Location = p.parts[-2].upper()
        self.T = datetime.strptime(p.stem, '%Y-%m')
        self.Year = self.T.year
        self.Tag = self.T.strftime('%Y%m')
        self.Name = self.T.strftime('%Y-%m')

    def __str__(self):
        return self.T.strftime('%b %Y')

    def __repr__(self):
        return f'Beam Test in {self} at {self.Location}'

In [ ]:
#| exports
def load_config():
    """A utility function that loads config file
    it expects main.ini file in the config directory under the root analyis directory
    Returns the instance of `Config` class"""
    config_file_path = Dir.joinpath('config', 'main.ini')
    if not isfile(config_file_path):
        warning('The main config file "config/main.ini" does not exist! Using the default!')
        copyfile(Dir.joinpath('config', 'default.ini'), config_file_path)
    return Config(config_file_path)

In [ ]:
#| export
class Analysis:
    """ The analysis class provides default behaviour objects in the analysis framework and is the parent of all other analysis objects. 
    The main part"""
    # Analysis Class Variables
    Config = load_config()
    Locations = Config.get_value('data', 'beam tests', type)
    DataDir = Path(Config.get('data', 'dir')).expanduser()
    ResultsDir = Dir.joinpath('results')
    MetaDir = Dir.joinpath(Config.get('SAVE', 'meta directory'))

    def __init__(self, 
                 beamtest:str=None, # A year and a month of the beam test, for example '201912' for DESY and '201810' for CERN
                 meta_sub_dir:str='', # Subdirectory for storing meta files
                 verbose:bool=False # Verbosity
                ):

        self.Verbose = verbose

        self.BeamTest = self.load_test_campaign(beamtest)
        self.MetaSubDir = meta_sub_dir

        self.Draw = SaveDraw(self, results_dir=self.BeamTest.Tag)

    def __str__(self):
        return f'{self.__class__.__name__.replace("Analysis", "").upper()} ANALYSIS'.strip(' ')

    def __repr__(self):
        return f'{self} ({self.BeamTest})'

    # ----------------------------------------
    # region INIT
    @property
    def server_save_dir(self):
        return



    @staticmethod
    def load_test_campaign(beamtest=None):
        bt = choose(beamtest, Analysis.Config.get('data', 'default test campaign')).replace('-', '')
        Analysis.init_locations()
        ps = [p for loc in Analysis.Locations for p in Path(Analysis.DataDir, loc.lower()).glob('*')]
        p = next((p for p in ps if bt == p.stem.replace('-', '')), None)
        return BeamTest(p)

    @staticmethod
    def find_testcampaign():
        """Determine the Tag of the test beam, either from the current path or from config file"""
        p = Path(getcwd())
        return BeamTest(p).Tag if p.parts[-2].upper() in Analysis.Locations else Analysis.Config.get('data', 'default test campaign')
    
    @staticmethod
    def init_locations():
        """creates folders for each beamtest organizing them by location then by date, 
        in order to store there raw and processed data storing raw and processed data"""
        for loc, beam_tests in Analysis.Locations.items():
            p = Analysis.DataDir.joinpath(loc.lower())
            p.mkdir(exist_ok=True)
            for bt in beam_tests:
                bt = str(bt)
                p.joinpath(f'{bt[:4]}-{bt[4:]}').mkdir(exist_ok=True)
    
    def print_testcampaign(self):
        """Prints current timestamp and the location and date of the beam test"""
        self.info(f'{self.BeamTest!r}')
    # endregion INIT
    # ----------------------------------------

    def info(self, msg, blank_lines=0, endl=True, prnt=None):
        return info(msg, blank_lines, endl, choose(prnt, self.Verbose))

    def add_info(self, t, msg='Done', prnt=None):
        add_to_info(t, msg, prnt=choose(prnt, self.Verbose))

    def make_pickle_path(self, name='', suf='', sub_dir=None, run=None, dut=None, camp=None):
        directory = self.MetaDir.joinpath(self.MetaSubDir if sub_dir is None else sub_dir)
        ensure_dir(directory)
        campaign = choose(camp, self.BeamTest.T.strftime('%Y%m'))
        dut = str(dut if dut is not None else self.DUT.Number if hasattr(self, 'DUT') and hasattr(self.DUT, 'Number') else '')
        run = choose(run, self.run_str)
        return directory.joinpath(f'{"_".join([v for v in [name, campaign, run, dut, str(suf)] if v])}.pickle')

    @property
    def unit_str(self):
        return f'run {self.Run}' if hasattr(self, 'Run') else ''

    @property
    def run_str(self):
        return str(self.Run) if hasattr(self, 'Run') else ''

    def get_meta_files(self):
        return [*Path(self.MetaDir).rglob(f'*_{self.BeamTest.Tag}_{self.run_str}*')] if self.run_str else []

    @property
    def meta_file_size(self):
        return sum(p.stat().st_size for p in self.get_meta_files())

    def print_meta_file_size(self):
        info(f'total size of metadata: {byte2str(self.meta_file_size)}')

    def print_meta_file_sizes(self):
        for p in self.get_meta_files():
            info(f'{p}: {byte2str(p.stat().st_size)}')
        self.print_meta_file_size()

    def remove_metadata(self):
        s = self.meta_file_size
        for p in self.get_meta_files():
            remove_file(p)
        info(f'removed {byte2str(s)} of meta files')

    def make_hdf5_path(self, *args, **kwargs):
        return self.make_pickle_path(*args, **kwargs).with_suffix('.hdf5')

    def print_start(self):
        print_banner(f'STARTING {self!r}', symbol='~', color=GREEN)

In [ ]:
#| export 
@patch
def create_run_config(self:Analysis):
    "Creates a runlog.json from Google spreadsheet"
    if self.BeamTest.Location == 'CERN':
        from src.spreadsheet import make_cern_run_log
        make_cern_run_log(self.BeamTest.Path.stem)
    elif self.BeamTest.Location == 'DESY':
        from src.spreadsheet import make_desy_run_log
        make_desy_run_log()

In [ ]:
#| export
@call_parse
def main():
    z = Analysis()

In [ ]:
#| hide
from nbdev import *
nbdev_export()

In [ ]:
show_doc(Analysis.init_locations)

---

[source](https://github.com/dmitryhits/HighResAnalysis/blob/master/HighResAnalysis/src/analysis.py#L106){target="_blank" style="float:right; font-size:smaller"}

### Analysis.init_locations

>      Analysis.init_locations ()

creates folders for each beamtest organizing them by location then by date, 
in order to store there raw and processed data storing raw and processed data

In [ ]:
show_doc(Analysis.find_testcampaign)

---

[source](https://github.com/dmitryhits/HighResAnalysis/blob/master/HighResAnalysis/src/analysis.py#L100){target="_blank" style="float:right; font-size:smaller"}

### Analysis.find_testcampaign

>      Analysis.find_testcampaign ()

Determine the Tag of the test beam, either from the current path or from config file

In [ ]:
Analysis.find_testcampaign()

'201912'

In [ ]:
show_doc(Analysis.print_testcampaign)

---

[source](https://github.com/dmitryhits/HighResAnalysis/blob/master/HighResAnalysis/src/analysis.py#L116){target="_blank" style="float:right; font-size:smaller"}

### Analysis.print_testcampaign

>      Analysis.print_testcampaign ()

Prints current timestamp and the location and date of the beam test

In [ ]:
z = Analysis('201912', verbose=True)
z.print_testcampaign()

INFO:     14:47:47 --> Beam Test in Dec 2019 at DESY


In [ ]:
Analysis.Locations

{'CERN': [201809, 201810], 'DESY': [201912]}